In [34]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K
from sklearn.utils import shuffle


## Load Data

In [35]:
dataset = pd.read_csv('datasets/Fs2_B_DANCE_WALK_KIN_0.5sec.csv')

### Remove Emotions from dataset & Split into Train/Test sets

In [36]:
dataset = dataset.drop(columns=['EMOTION_P', 'EMOTION_A', 'EMOTION_D'])

train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

train_dataset = np.asarray(train_dataset)
test_dataset = np.asarray(test_dataset)

x_train = train_dataset.reshape((len(train_dataset), np.prod(train_dataset.shape[1:])))
x_test = test_dataset.reshape((len(test_dataset), np.prod(test_dataset.shape[1:])))

print(len(x_train[0]))

No Training Samples: 39539
No Test Samples: 9885
20


## Build Model

### Settings

In [37]:
latent_dim = 5
intermediate_dim = 10
input_shape = (dataset.shape[1],)
print(input_shape)

(20,)


### Sampler

In [38]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        
        batch = tf.shape(z_mean)[0]
        
        dim = tf.shape(z_mean)[1]
        
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    
def sample_point_from_normal_distribution(args):
    mu, log_variance = args
    epsilon = K.random_normal(shape=K.shape(mu), mean=0.0, stddev=1.0)
    sampled_point = mu + K.exp(log_variance / 2) * epsilon
    return sampled_point

### Encoder

In [39]:
encoder_inputs = keras.Input(shape=input_shape)
x = layers.Dense(intermediate_dim, activation="relu")(encoder_inputs)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

z = Sampling()([z_mean, z_log_var])

encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 dense_9 (Dense)                (None, 10)           210         ['input_7[0][0]']                
                                                                                                  
 z_mean (Dense)                 (None, 5)            55          ['dense_9[0][0]']                
                                                                                                  
 z_log_var (Dense)              (None, 5)            55          ['dense_9[0][0]']                
                                                                                            

### Decoder

In [28]:
latent_inputs = keras.Input(shape=(latent_dim,))

x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)

decoder_outputs = layers.Dense(input_shape[0], activation="linear")(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense_7 (Dense)             (None, 10)                60        
                                                                 
 dense_8 (Dense)             (None, 20)                220       
                                                                 
Total params: 280
Trainable params: 280
Non-trainable params: 0
_________________________________________________________________


### Define VAE as a model with a custom train_step

In [29]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction)
                )
            )
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


## Train VAE

In [30]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(dataset, epochs=30, batch_size=128)

Epoch 1/30
387/387 [==============================] - 1s 1ms/step - loss: 374.4097 - reconstruction_loss: 304.1058 - kl_loss: 0.2233
Epoch 2/30
387/387 [==============================] - 0s 1ms/step - loss: 159.5041 - reconstruction_loss: 137.2085 - kl_loss: 0.5345
Epoch 3/30
387/387 [==============================] - 0s 1ms/step - loss: 97.8469 - reconstruction_loss: 90.7043 - kl_loss: 0.7474
Epoch 4/30
387/387 [==============================] - 0s 1ms/step - loss: 81.1014 - reconstruction_loss: 78.2794 - kl_loss: 0.9209
Epoch 5/30
387/387 [==============================] - 0s 1ms/step - loss: 74.3206 - reconstruction_loss: 71.3531 - kl_loss: 1.1814
Epoch 6/30
387/387 [==============================] - 0s 1ms/step - loss: 69.7042 - reconstruction_loss: 68.4300 - kl_loss: 1.1861
Epoch 7/30
387/387 [==============================] - 0s 1ms/step - loss: 69.4697 - reconstruction_loss: 66.3645 - kl_loss: 1.1397
Epoch 8/30
387/387 [==============================] - 0s 1ms/step - loss: 64.90

### Test

In [31]:
sample = np.asarray(dataset.iloc[1])
sample = sample.reshape(1,-1)
print(sample)

[[ 0.43891064  0.26525414  0.22958809  0.28302481  0.46816296  0.45200459
   0.35045947  0.3334689   0.286151    0.27875033  0.01188453 -0.14239762
  -0.06731151  0.98744555  0.09445444  0.01411131  0.01639102  0.02076603
   0.01443551  0.0214161 ]]


In [32]:
z_mean, z_log_var, z = vae.encoder.predict(sample)

In [33]:
regen = vae.decoder.predict(z)
print(regen)

[[ 0.40596476  0.34402603  0.33477613  0.3557668   0.38091022  0.36383072
   0.30834082  0.33207792  0.27965596  0.25482643 -0.2951659  -0.07551573
  -0.18715452  0.7797067   0.2665775   0.25822058  0.31572473  0.20905638
   0.23477826  0.23987621]]


In [10]:
def vae_loss(input_vols, output_vols):
    beta = 1e-7
    kl_loss = tf.keras.backend.sum(-1 - tf.keras.backend.log(tf.keras.backend.exp(z_log_var)) + tf.keras.backend.exp(z_log_var) + tf.keras.backend.square(z_mean))/2
    return tf.keras.backend.mean((input_vols-output_vols)**2) + beta*kl_loss

In [220]:
class VAE:
    def __init__(self, encoder, decoder, encoder_input, z_mean, z_log_var):
        self.encoder = encoder
        self.decoder = decoder
        
        self._model_input = encoder_input
        self.mu = z_mean
        self.log_variance = z_log_var
        
        self.model = None
        self._build_autoencoder()
        
        self.reconstruction_loss_weight = 1000
        
    def summary(self):
        self.encoder.summary()
        self.decoder.summary()
        self.model.summary()
        
    def compile(self, learning_rate=0.0001):
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer,
                          loss=self._calculate_combined_loss,
                          metrics=[self._calculate_reconstruction_loss,
                                   self._calculate_kl_loss])
        
    def train(self, train_set, batch_size, num_epochs):
        self.model.fit(train_set, train_set,
                      batch_size = batch_size,
                      epochs=num_epochs,
                      shuffle=True)
        
    def reconstruct(self, samples):
        latent_representations = self.encoder.predict(samples)
        reconstructed = self.decoder.predict(latent_representations)
        return reconstructed, latent_representations
    

    
    def _build_autoencoder(self):
        model_output = self.decoder(self.encoder(self._model_input))
        self.model = keras.Model(self._model_input, model_output, name="AutoEncoder")
        return
    
    def _calculate_combined_loss(self, y_target, y_predicted):
        reconstruction_loss = self._calculate_reconstruction_loss(y_target, y_predicted)
        kl_loss = self._calculate_kl_loss(y_target, y_predicted)
        print("HIIIIIIIIIIII")

        combined_loss = (self.reconstruction_loss_weight * reconstruction_loss) + kl_loss
        print(combined_loss)
        return combined_loss
    
    def _calculate_reconstruction_loss(self, y_target, y_predicted):
        error = y_target - y_predicted
        reconstruction_loss = K.mean(K.square(error), axis=1) # this axis may be wrong
        
        return reconstruction_loss
    
    def _calculate_kl_loss(self, y_target, y_predicted):
        kl_loss = -0.5 * K.sum(1 + self.log_variance - K.square(self.mu) - K.exp(self.log_variance), axis=1)
        return kl_loss